In [256]:
import requests
import csv
import pandas as pd
import re
import functools
import numpy as np
import warnings
from itertools import chain

warnings.simplefilter("ignore")

#### Code for generating month-wise data file for easy merge

In [173]:
data = []
url = 'https://ranking.ittf.com/public/s/ranking/csv?category=SEN&typeGender=M%3BSINGLES&year=2018&month={}'

for i in range(1,9):
    response = requests.get(url.format(i))
    
    for resp in response.content.decode("utf-8").splitlines():
        resp = resp.split(',')
        data.append(resp)
        
    with open("data_2018_{}.csv".format(i), "w") as csv_file:
        writer = csv.writer(csv_file)
        for line in data:
            writer.writerow(line)
    data = []

In [371]:
### Fetching all month files and merging them into a single dataframe on unique column "ID" using reduce method.
file_names = ["data_2018_{}.csv".format(i) for i in range(1,10)]
df_list = [pd.read_csv(file_name) for file_name in file_names]
data = functools.reduce(lambda x, y: pd.merge(x, y, on = 'ID'), df_list)

In [372]:
### Function for removing unwanted columns
def drop_unwanted_cols(data,*args):
    for arg in args:
        column_names = [col for col in data.columns for arg in args if col.startswith(arg)]      
    new_data = data.drop(column_names,axis=1)
    return new_data

In [373]:
## Copying column values to be merged later into the data
gender = data['Gender_x'].values
name = data['Name_x'].values
country = data['Assoc_x'].values
year = data['YearNum_x'].values
ids = data['ID'].values

In [374]:
# Dropping unwanted columns
data = drop_unwanted_cols(data,'Previous','Gender','Name','Assoc','Year','WeekNum','MonthNum')
data = data.drop('ID',axis =1)

In [375]:
## Code for creating column names in month order
unique_cols = ['Rank','Points']
all_cols = ["{}_{}".format(col,i) for col in unique_cols for i in range(1,10)]

rank_names = all_cols[0:9]
points_names = all_cols[9:18]

new_columns = [rank_names[i]+" "+points_names[i] for i in range(0,9)]
new_list = [nc.split(' ') for nc in new_columns]
new_cols = list(chain.from_iterable(new_list))

In [376]:
data.columns = new_cols

In [377]:
data['ID'] = data['Name'] = data['Gender'] = data['Country'] = data['Year'] = np.nan

In [378]:
data['ID'] = ids
data['Name'] = name
data['Gender'] = gender
data['Country'] = country
data['Year'] = year

In [379]:
columns = data.columns.tolist()
cols = columns[18:24] + columns[0:18]

In [382]:
data[cols].to_csv('ITTF_Ranking_2018.csv')